# 1. 크롤링


- 점핏에서 채용공고 url들 가져오기
- 점핏 채용 공고 크롤링
- 사람인 기업 정보 링크 리스트 만들기
- 사람인 기업 정보 크롤링
- Gensim 이용하여 qulification, more 전체 요약
- Chat GPT Open API 활용 방안

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import requests

from datetime import datetime

# 1-1. 점핏에서 채용공고 url들 가져오기


![jumpit.png](img/jumpit.png)



## 1) 원하는 직종 검색


```
keyword = input("검색어를 입력하세요:")
```


이번 발표에서는 '**데이터 엔지니어**' 검색



## 2) 셀레니움으로 브라우저 실행


```
driver = ChromeDriverManager().install()
service = Service(executable_path=driver)
browser = webdriver.Chrome(service=service)

browser.maximize_window()
```


## 3) 키워드 검색


원래는 점핏 페이지의 검색창을 찾아 직접 입력하는 식으로 진행하려 했으나,
점핏에서는 검색 페이지 url에 일정한 패턴 존재함을 발견.
이 패턴을 이용하여 바로 원하는 키워드 검색 페이지로 이동


```
browser.get(f'https://www.jumpit.co.kr/search?sort=relation&keyword={keyword}')
```


## 4) 무한 스크롤


```
scroll_pane_height = browser.execute_script("return document.documentElement.scrollHeight")
while True:
    # scroll 이동 - window.scrollTo(가로스크롤이동할위치, 세로스크롤이동할위치)
    browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
    time.sleep(1)
    # 이동후 height길이 조회
    new_scroll_height = browser.execute_script("return document.documentElement.scrollHeight")
    if scroll_pane_height == new_scroll_height:
        break #  스크롤 이동을 멈춘다.
    scroll_pane_height = new_scroll_height
```


## 5) 채용공고 페이지 url 리스트 만들기


![url.png](img/url.png)


```
#url 수집
urls = browser.find_elements(By.CSS_SELECTOR, "section > div > a")


url_list=[]
for url in urls:
    url = url.get_attribute("href")
    url_list.append(url)
```

In [ ]:
#검색어 입력
keyword = input("검색어를 입력하세요:")

# 브라우저 실행
driver = ChromeDriverManager().install()
service = Service(executable_path=driver)
browser = webdriver.Chrome(service=service)

browser.maximize_window()

# 검색
browser.get(f'https://www.jumpit.co.kr/search?sort=relation&keyword={keyword}')

# 무한 스크롤
scroll_pane_height = browser.execute_script("return document.documentElement.scrollHeight")
while True:
    # scroll 이동 - window.scrollTo(가로스크롤이동할위치, 세로스크롤이동할위치)
    browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight)")
    time.sleep(1)
    # 이동후 height길이 조회
    new_scroll_height = browser.execute_script("return document.documentElement.scrollHeight")
    if scroll_pane_height == new_scroll_height:
        break #  스크롤 이동을 멈춘다.
    scroll_pane_height = new_scroll_height


#url 수집
urls = browser.find_elements(By.CSS_SELECTOR, "section > div > a")


url_list=[]
for url in urls:
    url = url.get_attribute("href")
    url_list.append(url)
    
print(url_list)
print('url 갯수:', len(url_list))

# 브라우저 종료
browser.close()

# 1-2. 점핏 채용 공고 크롤링


## 1) 크롤링 빈 리스트 생성


```
company_list = []          # 회사명
title_list = []            # 공고 제목(주로 직업명)
city_list = []          # 시
district_list = []      # 구
remain_address_list = []  # 나머지 주소
career_list = []           # 요구 경력
education_date_list = []   # 요구 학력
stack_list = []            # 요구 기술
main_task_list = []        # 주요 업무
qualification_list = []    # 자격 요건
more_list = []             # 우대 사항
duetime_list = []          # 마감 시한
```


## 2) requests


정보를 얻기 위해 '더보기 버튼' 혹은 무한 스크롤등 필요 없는 페이지.
위에서 얻은 url 리스트를 for문으로 하나씩 크롤링


```
for link in url_list:
    
    url = link
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    
    # 1. 요청
    res = requests.get(url, headers={"User-Agent": user_agent})
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
```


## 3) company, title


![company.png](img/company.png)


```
# company
companies = soup.select('div > section.sc-3a5c981a-0.iORPxB > div > a')

for company in companies:
    company = company.text
    company_list.append(company)
    
    
# title
titles = soup.select('div > section.sc-3a5c981a-0.iORPxB > h1')

for title in titles:
    title = title.text
    title_list.append(title)
```


## 4) place, career, duetime


![career.png](img/career.png)


**html**

```
<section class="sc-2da322c6-0 hWFbtF"><h2 class="blind">포지션 경력/학력/마감일/근무지역 정보</h2><dl class="sc-2da322c6-1 ksYhFT"><dt>경력</dt><dd>경력 2~10년</dd></dl><dl class="sc-2da322c6-1 ksYhFT"><dt>학력</dt><dd>대학교졸업(4년) 이상</dd></dl><dl class="sc-2da322c6-1 ksYhFT"><dt>마감일</dt><dd>상시</dd></dl><dl class="sc-2da322c6-1 ksYhFT"><dt>근무지역</dt><dd><ul><li>서울시 서초구 강남대로 507, 6층<div class="btns"><div><a rel="noreferrer" target="_blank" title="점핏 앱에서는 네이버 지도보기 모바일 뷰로 페이지 이동합니다" class="sc-6b85ed6d-0 igMlQB" href="https://m.map.naver.com/search2/search.nhn?query=서울시 서초구 강남대로 507, 6층#/map">지도보기</a><span aria-hidden="true" class="sc-6b85ed6d-1 hIEzpC">·</span><button type="button" class="sc-169b37f3-9 giFusE">주소복사</button></div></div></li></ul></dd></dl></section>

```


```
# place
place_tag = soup.find('dt', text='근무지역')  # 근무지역을 나타내는 dt 태그 찾기
place = place_tag.find_next_sibling('dd').text.strip()  # dd 태그에서  추출
place = place.replace("지도보기·주소복사", "")  # 불필요한 부분 제거
    
# 공백을 기준으로 시, 구, 전체 주소를 나눔
place_parts = place.split(' ')
city = place_parts[0]  # 첫 번째 요소는 '시'
district = place_parts[1]  # 두 번째 요소는 '구'
city_district = place_parts[0:1]
full_address = ' '.join(place_parts[2:])  # 세 번째 요소부터는 전체 주소로 합침
    
# 각각의 리스트에 추가
city_list.append(city)
district_list.append(district)
remain_address_list.append(full_address)

            
            
# career
careers = soup.select('div > section.sc-2da322c6-0.hWFbtF > dl:nth-child(2) > dd')
        
    for career in careers:
        career = career.text
        career_list.append(career)

# duetime
duetimes = soup.select('section.sc-2da322c6-0.hWFbtF > dl:nth-child(4) > dd')
        
     for duetime in duetimes:
         duetime = duetime.text
         duetime_list.append(duetime)
```


## 5) main task, qualification, more


![task.png](img/task.png)


```
# main_task
main_tasks = soup.select('section.position_info > dl:nth-child(3) > dd > pre')
        
   for main_task in main_tasks:
       main_task = main_task.text
       main_task_list.append(main_task)
            
            
# qualification
qualifications = soup.select('section.position_info > dl:nth-child(4) > dd > pre')
        
    for qualification in qualifications:
        qualification = qualification.text
        qualification_list.append(qualification)
            
            
# more
mores = soup.select('section.position_info > dl:nth-child(5) > dd > pre')

      for more in mores:
          more = more.text
          more_list.append(more)
```


## 6) stack


![stack.png](img/stack.png)


**개별 스택 html**

```
<div class="sc-c8fd94d7-0 gChXoq"><img alt="MongoDB" height="20" src="https://cdn.jumpit.co.kr/images/stacks/mongodb.png" width="20">MongoDB</div>
```


```
# stack 
stacks = soup.select('dl:nth-child(2) > dd > pre')
   for stack in stacks:
    # 이미지 태그에서 alt 속성 추출
    alt_attributes = [tag.img['alt'] for tag in soup.find_all 
                       ('div', class_='sc-c8fd94d7-0 gChXoq')]
    # 쉼표로 구분된 문자열로 변환
    result = ", ".join(alt_attributes)
    stack_list.append(result)
```

In [ ]:
# 기업 정보 페이지 이동 링크 리스트

company_list = []          # 회사명
title_list = []            # 공고 제목(주로 직업명)
city_list = []          # 시
district_list = []      # 구
remain_address_list = []  # 나머지 주소
career_list = []           # 요구 경력
education_date_list = []   # 요구 학력
stack_list = []            # 요구 기술
main_task_list = []        # 주요 업무
qualification_list = []    # 자격 요건
more_list = []             # 우대 사항
duetime_list = []          # 마감 시한

for link in url_list:
    
    url = link
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    
    # 1. 요청
    res = requests.get(url, headers={"User-Agent": user_agent})
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        
        # company
        companies = soup.select('div > section.sc-3a5c981a-0.iORPxB > div > a')
        
        for company in companies:
            company = company.text
            company_list.append(company)
            
            
        # title
        titles = soup.select('div > section.sc-3a5c981a-0.iORPxB > h1')
        
        for title in titles:
            title = title.text
            title_list.append(title)
            
            
        # place
        place_tag = soup.find('dt', text='근무지역')  # 근무지역을 나타내는 dt 태그 찾기
        place = place_tag.find_next_sibling('dd').text.strip()  # dd 태그에서 근무지역 텍스트 추출
        place = place.replace("지도보기·주소복사", "")  # 불필요한 부분 제거
    
        # 공백을 기준으로 시, 구, 전체 주소를 나눔
        place_parts = place.split(' ')
        city = place_parts[0]  # 첫 번째 요소는 '시'
        district = place_parts[1]  # 두 번째 요소는 '구'
        city_district = place_parts[0:1]
        full_address = ' '.join(place_parts[2:])  # 세 번째 요소부터는 전체 주소로 합침
    
        # 각각의 리스트에 추가
        city_list.append(city)
        district_list.append(district)
        remain_address_list.append(full_address)

            
            
        # career
        careers = soup.select('div > section.sc-2da322c6-0.hWFbtF > dl:nth-child(2) > dd')
        
        for career in careers:
            career = career.text
            career_list.append(career)
            
            
        # education_date
        education_dates = soup.select('div > section.sc-2da322c6-0.hWFbtF > dl:nth-child(3) > dd')
        
        for education_date in education_dates:
            education_date = education_date.text
            education_date_list.append(education_date)
            
            
        # stack 
        stacks = soup.select('dl:nth-child(2) > dd > pre')
        for stack in stacks:
            # 이미지 태그에서 alt 속성 추출
            alt_attributes = [tag.img['alt'] for tag in soup.find_all('div', class_='sc-c8fd94d7-0 gChXoq')]
            # 쉼표로 구분된 문자열로 변환
            result = ", ".join(alt_attributes)
            stack_list.append(result)

            
        # main_task
        main_tasks = soup.select('section.position_info > dl:nth-child(3) > dd > pre')
        
        for main_task in main_tasks:
            main_task = main_task.text
            main_task_list.append(main_task)
            
            
        # qualification
        qualifications = soup.select('section.position_info > dl:nth-child(4) > dd > pre')
        
        for qualification in qualifications:
            qualification = qualification.text
            qualification_list.append(qualification)
            
            
        # more
        mores = soup.select('section.position_info > dl:nth-child(5) > dd > pre')
        
        for more in mores:
            more = more.text
            more_list.append(more)
            
            
        # duetime
        duetimes = soup.select('section.sc-2da322c6-0.hWFbtF > dl:nth-child(4) > dd')
        
        for duetime in duetimes:
            duetime = duetime.text
            duetime_list.append(duetime)
            

print("comapny_list: ", len(company_list))
print("title_list: ", len(title_list))
print("city_list: ", len(city_list))
print("district_list: ", len(district_list))
print("remain_address_list: ", len(remain_address_list))
print("career_list: ", len(career_list))
print("education_date_list: ", len(education_date_list))
print("stack_list: ", len(stack_list))
print("main_task_list: ", len(main_task_list))
print("qualification_list: ", len(qualification_list))
print("more_list: ", len(more_list))
print("duetime_list: ", len(duetime_list))


browser.close()

# 1-3. 사람인 기업 정보 링크 리스트 만들기


위에서 만든 company_list를 활용하여 
사람인 주소 창에 회사를 검색하는 url f스트링 형성

In [ ]:
# 기업 정보 페이지 이동 링크 리스트
link_list = []

for company in company_list:
    
    url = f'https://www.saramin.co.kr/zf_user/search/company?search_area=main&search_done=y&search_optional_item=n&searchType=search&searchword={company}'
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    
    # 1. 요청
    res = requests.get(url, headers={"User-Agent": user_agent})
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        
        tag_list = soup.select_one('div.content > div > h2 > a')
        if tag_list:
            link_list.append(tag_list.get("href"))
            
        else:
            link_list.append('NoN')
    
print(link_list)

# 1-4. 사람인 기업 정보 크롤링


## 1. 사람인에 기업 정보가 없을 경우 대비


```
for link in link_list:
    
    if link != 'NoN':
    
    ----- 크롤링 코드 -----
    
    else:
        years_of_experience_list.append(None)
        established_date_list.append(None)
        industry_list.append(None)
        business_description_list.append(None)
        total_employees_list.append(None)
        company_type_list.append(None)
        view_list.append(None)
```


## 2. 크롤링


![saram1.png](img/saram1.png)

![saram2.png](img/saram2.png)

In [ ]:
from bs4 import BeautifulSoup

# 결과를 저장할 리스트 초기화
years_of_experience_list = []
established_date_list = []
industry_list = []
business_description_list = []
total_employees_list = []
company_type_list = []
view_list = []

# 링크 리스트 순회
for link in link_list:
    
    if link != 'NoN':
        url = 'https://www.saramin.co.kr' + link
        user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
        
        # 요청 보내기
        res = requests.get(url, headers={"User-Agent": user_agent})
        if res.status_code == 200:
            
            # HTML 파싱
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # 회사 요약 정보 가져오기
            company_summary = soup.find('ul', class_='company_summary')
            if company_summary:
                # 업력 정보 추출
                years_of_experience_tag = company_summary.find('strong', class_='company_summary_tit')
                years_of_experience_text = years_of_experience_tag.text.strip() if years_of_experience_tag else None
                
                # "업력"과 "차"를 제거하고 숫자만 추출
                if years_of_experience_text:
                    years_of_experience = ''.join(filter(str.isdigit, years_of_experience_text))
                else:
                    years_of_experience = None
                
                # 설립일 정보 추출
                established_date_tag = company_summary.find('p', class_='company_summary_desc')
                established_date = established_date_tag.text.strip() if established_date_tag else None
                
                # 날짜 형식으로 변환
                if established_date:
                    established_date = established_date.replace('설립', '').strip()
                    # 날짜 포맷 변경 (예: "2018년 11월 20일" -> "2018-11-20")
                    established_date = established_date.replace('년 ', '-').replace('월 ', '-').replace('일', '')
                else:
                    established_date = None
                
                # 업종 가져오기
                industry_tag = soup.find('dt', text='업종')
                industry = industry_tag.find_next_sibling('dd').text.strip() if industry_tag else None
                
                # 사업내용 가져오기
                business_description_tag = soup.find('dt', text='사업내용')
                business_description = business_description_tag.find_next_sibling('dd').text.strip() if business_description_tag else None
            
                # 전체 사원수 가져오기
                total_employees_tag = soup.find('span', class_='tit', text='전체 사원수')
                if total_employees_tag:
                    next_sibling = total_employees_tag.find_next_sibling('span', class_='value')
                    if next_sibling:
                        total_employees_text = next_sibling.find('strong', class_='num').text.strip()
                        total_employees = ''.join(filter(str.isdigit, total_employees_text))
                    else:
                        total_employees = None
                else:
                    total_employees = None
                
                # 기업 형태 추출
                company_type_text = None

                # TipTxt 클래스를 가진 div 요소 중에서 기업형태를 찾기
                for tip_txt_div in company_summary.find_all('div', class_='TipTxt'):
                    if '기업형태' in tip_txt_div.text:
                        company_type_text = tip_txt_div.text.split(':')[-1].strip()
                        break

                company_type_list.append(company_type_text)
                
                # 조회수 가져오기
                favor_company = soup.find('div', class_='favor_company')
                view_tag = favor_company.find('span', class_='tit', text='조회수')
                view = view_tag.find_next_sibling('div', class_='counting').find('strong', class_='num').text.strip() if view_tag else None
                view_list.append(view)
                
            else:
                years_of_experience = None
                established_date = None
                industry = None
                business_description = None
                total_employees = None
                company_type_list.append(None)
                view_list.append(None)
                
            # 각 리스트에 값 추가
            years_of_experience_list.append(years_of_experience)
            established_date_list.append(established_date)
            industry_list.append(industry)
            business_description_list.append(business_description)
            total_employees_list.append(total_employees)
            
    else:
        years_of_experience_list.append(None)
        established_date_list.append(None)
        industry_list.append(None)
        business_description_list.append(None)
        total_employees_list.append(None)
        company_type_list.append(None)
        view_list.append(None)


In [ ]:
import pandas as pd
df = pd.DataFrame({'company':company_list, 
                  'title':title_list,
                  'url': url_list, 
                  'city':city_list,
                  'district':district_list,
                  'remain_address':remain_address_list, 
                  'career':career_list, 
                  'education_date':education_date_list,
                  'stack':stack_list,
                  'main_task': main_task_list, 
                  'qualification':qualification_list,
                  'more':more_list,
                  'duetime':duetime_list,
                  'years_of_experience':years_of_experience_list,
                  'established_date':established_date_list,
                  'total_employees':total_employees_list,
                  'company_type':company_type_list,
                  'view':view_list})

filename = f"jumpit_crawling_{keyword}_{datetime.now().strftime('%Y-%m-%d')}.csv"
df.to_csv(filename)  # 파일로 저장 -> csv 포멧으로 저장.
# CSV 파일을 DataFrame으로 읽기
df = pd.read_csv(filename)
df

In [1]:
import pandas as pd

df = pd.read_csv('jumpit_crawling_데이터 엔지니어_2024-02-19.csv', index_col='Unnamed: 0')
df

,company,title,url,city,district,remain_address,career,education_date,stack,main_task,qualification,more,duetime,years_of_experience,established_date,total_employees,company_type,view
0,워트인텔리전스,[워트 개발팀] 데이터 엔지니어 모집 (국내 최대 데이터),https://www.jumpit.co.kr/position/7326,서울시,서초구,"강남대로 507, 6층",경력 2~10년,대학교졸업(4년) 이상,"MongoDB, BigData, REST API, Hadoop, Java, Luce...","다양한 서비스를 제공하기 위해 Airflow를 활용한 데이터 파이프라인 구축, 데이...","• Java, Python, SQL 을 다루는데 어려움이 없으신 분\n• Linux...","• 검색서비스 개발과 자연어 처리 경험 보유자 우대\n• 머신러닝, 딥러닝 기반 업...",상시,10.0,2015-5-27,40.0,"중소기업, 주식회사, 사단법인",370
1,핀다,오픈업 데이터 엔지니어 (Python),https://www.jumpit.co.kr/position/22641,서울,강남구,"테헤란로 518, 4층 (대치동,한국섬유산업연합회)",경력 3~10년,대학교졸업(4년) 이상,"Git, MongoDB, Amazon DynamoDB, FastAPI, Airflo...",[오픈업을 소개합니다.]\n핀다의 오픈업은 소상공인과 자영업자들의 금융 고민을 해결...,• Python을 활용한 데이터 처리 경험이 있는 분\n• 데이터 프로세싱을 위한 ...,• GIS를 활용한 데이터 구조화가 가능한 분\n• 지역 기반의 상권 및 매출 관련...,상시,10.0,2015-9-25,127.0,"스타트업, 중소기업, 병역특례 인증업체",544
2,무브멘토,AI 빅데이터 엔지니어 / 딥러닝 엔지니어 (빅데이터 실무자 모집),https://www.jumpit.co.kr/position/18717,서울,성동구,아차산로 104 무신사스튜디오 401호,신입,대학교졸업(4년) 이상,"AI/인공지능, BigData, C++, DeepLearning, NLP, Tens...",딥러닝 엔지니어\n• 딥러닝 영상처리 알고리즘 개발 (페이스스왑 기술 관련)\n• ...,"• 학력: 학사 이상\n• 경력 신입 가능\n• 파이썬/C/C++및 영상처리, 음성...",• 대학원(석사) 이상\n• 관련 딥러닝 모델 연구/개발자 우대 (Tensor Fl...,상시,NaN,기업형태,NaN,NaN,400
3,웅진,데이터 엔지니어 (3년 이상),https://www.jumpit.co.kr/position/22009,서울시,중구,청계천로 24 케이스퀘어시티,경력 3~16년,무관,"QlikView, Java, Python, PowerBI, Microsoft Pow...","• Azure data Platform 을 활용한 빅데이터 수집, 저장, 처리, 분...",• 모집직급: 대리 ~ 과장 \n• MS Power Platform 구축 / 운영 ...,"• MS Azure 및 전산관련 자격증 보유 \n• 빅데이터 통계분석, DW&DM...",2024-03-29,42.0,1983-3-21,484.0,"중견기업, 코스피, 주식회사, 외부감사법인","1,092"
4,비포플레이,데이터 엔지니어,https://www.jumpit.co.kr/position/20944,서울,강남구,"테헤란로 217, 601호",경력 3~10년,무관,"Etl, NoSql, Java, Python","• 웹크롤링, 오픈API를 통한 데이터 수집\n• ETL 환경 구축 및 관리\n• ...",• 데이터 엔지니어링 및 인프라 개발/운영 경력이 3년 이상 이거나 이에 준하는 실...,• 분산처리기술 경험\n• 대용량 및 실시간 데이터 분산 처리 시스템 개발 경험\n...,상시,5.0,2020-7-17,18.0,"중소기업, 주식회사",107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,업스테이지,Software Engineer - Backend,https://www.jumpit.co.kr/position/20693,경기,용인시,수지구 광교중앙로 338 리모트근무,경력 3~15년,무관,"Golang, TypeScript, C++, NoSql, AWS, Python, C...",• Upstage의 핵심 웹 애플리케이션 제품의 백엔드 개발\n• 기계 학습 모델 ...,"• 하나 이상의 프로그래밍 언어에 대한 높은 수준의 이해 (예: Golang, Py...",• Python 또는 Golang 을 이용한 서버 개발 경험\n• 대규모 트래픽 처...,상시,5.0,2020-10-5,130.0,NaN,"1,210"
270,오엔제이,시니어 백엔드 개발자,https://www.jumpit.co.kr/position/20678,서울시,강남구,"역삼로 122, 202호",경력 5~10년,무관,"amazon ec2, Git, MySQL, spring, Redis, java","• 당사 서비스 플랫폼 (Weldbeing) 백엔드 개발 및 운영 \n• H/A, ...",• Spring/Spring Boot 프레임워크를 이용한 개발 5년 이상이신 분\n...,• 업무를 주도적으로 진행하고 열정적인 분\n• 더 나은 코드를 위해 항상 노력하고...,상시,4.0,사원수,4.0,NaN,32
271,안티그래비티,서비스 플랫폼 프론트엔드(front-end) 개발,https://www.jumpit.co.kr/position/8833,서울,성동구,성수이로22길 37 아크밸리 지식산업 센터 303c호 안티그래비티,경력 4~20년,무관,"JavaScript, Git, TypeScript, React, REST API, ...",✔ 담당하시게 될 업무는 'Front-end 개발' 입니다!\n• 자사 프론트엔드 ...,"✔ 어디 이런 분 없나요?\n• HTML JavaScript CSS, Vue/Rea...","✔ 이런 경험 가지신 분, 레드카펫 깔아드립니다.\n• 클라우드(AWS, Azure...",상시,7.0,2018-5-30,18.0,"중소기업, 주식회사",163
272,안티그래비티,"서비스 플랫폼 백엔드(back-end, spring boot) 개발",https://www.jumpit.co.kr/position/8834,서울,성동구,성수이로22길 37 아크밸리 지식산업 센터 303c호 안티그래비티,경력 4~20년,무관,"JavaScript, Git, TypeScript, REST API, Spring ...","✔ 담당하시게 될 업무는 'Back-end 개발' 입니다!\n• (JAVA, Spr...","✔ 어디 이런 분 없나요?\n• (JAVA, Spring Boot)를 이용한 웹 /...","✔ 이런 경험 가지신 분, 레드카펫 깔아드립니다.\n• 클라우드(AWS, Azure...",상시,7.0,2018-5-30,18.0,"중소기업, 주식회사",163


# 1-5. Gensim 이용하여 qulification, more 전체 요약

- Gensim 라이브러리

: 자연어를 벡터로 변환하는데 필요한 대부분의 편의 기능을 제공하고 있는 라이브러리

- Gensim 설치
```
!pip install gensim==3.4.0  
```

## 1. more

In [2]:
# 전체 항목을 한 문장으로 합치기
more = df['more']
combined_sentence = ' '.join(more)
combined_sentence

'• 검색서비스 개발과 자연어 처리 경험 보유자 우대\n• 머신러닝, 딥러닝 기반 업무 경험 또는 플랫폼 개발 경험 보유자 우대 • GIS를 활용한 데이터 구조화가 가능한 분\n• 지역 기반의 상권 및 매출 관련 데이터 분석에 관심이 있거나 관련 경험이 있는 분\n• 데이터 전처리 및 시각화 경험이 있는 분\n• 시스템 최적화와 성능 개선에 관심이 많은 분\n• Pandas 등 데이터 라이브러리 사용 경험이 있는 분\n• Apache Spark를 활용한 데이터 처리 경험이 있는 분 • 대학원(석사) 이상\n• 관련 딥러닝 모델 연구/개발자 우대 (Tensor Flow, PyTouch)\n• 알고리즘(모델)에 기반한 영상 처리 및 컴퓨터 그래픽 전공자 또는 경력자\n• 데이터 레이블링/라벨링 업무 경험자\n• 데이터 검수 경험자 • MS Azure  및 전산관련 자격증 보유 \n• 빅데이터 통계분석, DW&DM 구축, 다양한 시각화 설계 및 구현 경험  \n• Power BI, Tableau, Qlik, Cognos, Spotfire, MSTR, Longview 등 BI 솔루션 활용 또는 프로젝트 경험\n• API, JAVA, Node.js, Python 등 개발기술 경험  • 분산처리기술 경험\n• 대용량 및 실시간 데이터 분산 처리 시스템 개발 경험\n• Data Lake 개발/운영 경험\n• 데이터 분석가/사이언티스트 및 타 팀과의 커뮤니케이션이 원활하신분 • 헬스케어 관련 데이터에 대한 경험/이해가 있는 분\n• MLOps를 이용하여 실제 서비스를 운영 해본 경험이 있으신 분\n• 주도적으로 문제를 해결하고 개선하고자 하는 의지를 가진 분\n• 헬스케어 도메인에서 무언가를 이뤄보고 싶으신 분 • 관련분야 석사 이상의 학위, 혹은 3년 이상의 경력이 있으신 분\n• 머신러닝, 딥러닝을 활용한 모델 개발 및 모델의 실제 서비스 적용/운영 경험이 있는 분\n• NLP 경험 있으신 분 • LLM 서비스 구축 경험이 있으신 분\n• 백엔드 개발 경력 2년 이상 이

In [ ]:
# !pip install gensim==3.4.0  

In [3]:
from gensim.summarization.summarizer import summarize

text = summarize(combined_sentence, word_count=200)

# '•'를 기준으로 나누고 줄바꿈 적용
items = text.split('•')
# 각 항목을 정리하여 새로운 줄에 표시하고, 첫 항목에도 '•'를 추가
formatted_text = '•' + '\n•'.join(item.strip() for item in items if item.strip())

print(formatted_text)

C:\Users\Playdata\AppData\Local\anaconda3\Lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


•고가용성 시스템 설계, 구축 가능한 분 우대
•정형/비정형 데이터 처리 경험이 있으신 분
•R, Python, Pyspark Programming 경험이 있으신 분
•빠르게 성장하는 서비스에서의 고가용성 데이터 플랫폼 설계 및 운영 경험
•AWS, GCP 등의 클라우드 서비스 사용 경험이 있으신 분
•컴퓨터공학, 산업공학, 전기전자 관련 학위
•AWS(DynamoDB, EC2, RDS), GCP(BigQuery), Google Analytics 사용 경험이 있으신 분 [이런 분이면 더욱 좋아요!]
•기타 Python 기반의 프로젝트 경험이 있으신 분
•딥러닝 분야 석사 이상의 학위 또는 이에 준하는 경력을 보유하신 분
•AWS, Azure, GCP 등 클라우드 플랫폼 경험
•JIRA, Confluence 등 프로젝트 관리 툴 사용 경험이 있으신 분
•Jira, Confluence 등의 툴을 이용한 협업 및 커뮤니케이션 경험을 가지신 분
•클라우드 환경 내 MSA 기반 인프라 구축 경험이 있으신 분
•LLM fine-tuning경험, LLMOps 구축경험, LangChain 개발경험, CKA, CKS, CNCF, CNCF 에코시스템 개발경험, Golang서버개발경험, 빅데이터(Big Data) 에코시스템 개발경험, AI 파이프라인경험(MLOps, AIOps, CI/CD), AWS 등 클라우드 서비스경험, Database/API 개발경험, 데이터센터인프라개발경험, 블록체인 프로젝트 개발경험, AI모델개발 경험(캐글 및 주요대회 상위입상), AI 모델서빙 및 운용경험, 탑티어 컨퍼러스 및 저널 출판 경험, 보안AI(secure AI, federated learning)개발경험, 시계열데이터 AI 개발경험, 인과추론 AI 개발경험, MOOC(cs329s, cs231n, cs224n)
•컴퓨터/시스템공학 전공자
•대용량 데이터 구축, 처리 및 서비스 경험이 있으신 분


## 2. qulification 

In [13]:
qualification = df['qualification']
combined_sentence2 = ' '.join(qualification)
combined_sentence2

'• Java, Python, SQL 을 다루는데 어려움이 없으신 분\n• Linux 환경 하에 개발 및 시스템 운영 \n• 데이터 엔지니어 경력 2년 이상 • Python을 활용한 데이터 처리 경험이 있는 분\n• 데이터 프로세싱을 위한 백엔드 시스템을 개발하고 운영한 경험이 있는 분\n\n[핀다 오픈업 데이터 엔지니어가 사용하고 있는 기술]\n• Python, Pandas, Streamlit\n• MongoDB, SQL, AWS DynamoDB, S3\n• JS, GO, AWS Lambda, SQS, FastAPI\n• Git, GitLab CI, Airflow, Grafana • 학력: 학사 이상\n• 경력 신입 가능\n• 파이썬/C/C++및 영상처리, 음성처리, NLP\n• 라이브러리(ffmpeg,opencv, dlib등)\n• 사용 개발 경험\n• 유관 업무 경험자\n• 이미지 및 영상 관련 이해도가 높은분 • 모집직급: 대리 ~ 과장 \n• MS Power Platform 구축 / 운영 경험 3년 이상\n   (Automate, BI, Apps 중 1개 이상 경험 포함)\n• Power BI 대시보드 작성 능력 보유\n• 데이터에 대한 기본 지식 보유 및 Data business Initiative 도출할 수 있는 기획력\n• 데이터의 흐름을 이해하고 사고할 수 있는 분\n• Python을 활용한 프로그램 개발 가능자 • 데이터 엔지니어링 및 인프라 개발/운영 경력이 3년 이상 이거나 이에 준하는 실력을 가지신 분\n• Python, Java 등 프로그래밍 언어 스킬보유하신 분\n• RDB, NoSQL 등 데이터 소스 활용을 실무에 적용해보신분\n• 클라우드 환경에서 데이터 엔지니어링 경험이 있으신분\n• 사용하는 기술의 원리를 알고 실무에 적용해보신 분 • 데이터 분석 및 모델링 경력 2년 이상 또는 그에 준하는 역량을 보유하신 분\n• 대표적인 ML/DL 모델을 이해하고 특징 및 차이에 대해 이해하고 계신 분\n• Python, SQL, R 등의

In [14]:
text = summarize(combined_sentence2, word_count=200)

# '•'를 기준으로 나누고 줄바꿈 적용
items = text.split('•')
# 각 항목을 정리하여 새로운 줄에 표시하고, 첫 항목에도 '•'를 추가
formatted_text = '•' + '\n•'.join(item.strip() for item in items if item.strip())

print(formatted_text)

•내가 제안한 설계와 개발한 코드에 대한 비판에도 열린 커뮤니케이션이 가능한 분
•PyTorch, Tensorflow 등을 이용한 ML 프로젝트 개발 경험이 있으신 분
•숙련된 SQL 능력과 Python 데이터 처리 경험을 보유한 분
•Data flow Architecture 설계가 가능하신 분
•데이터와 인사이트를 분석하여 명확하고 실행 가능한 계획을 제시한 경험이 있으신 분
•4년 이상의 데이터 엔지니어링 실무 경력이 있는 분
•SQL(BigQuery, PostgreSQL), Python 등을 활용한 데이터 분석 실무 경험이 있으신 분
•프로그래밍 언어(Python, R 등)에 대한 이해 및 사용 능력자
•데이터 파이프라인 구축 경험이 있으신 분
•딥러닝 프레임워크(Tensorflow, PyTorch 등)를 활용한 모델 개발 및 배포 경험이 있으신 분
•Kubernetes, Docker 등 Container 환경에 대한 지식을 보유하신 분 - 딥러닝 모델을 활용하여 응용 분야에 대한 연구 개발 경험이 있으신 분 - 해외 출장에 결격 사유가 없는 분
•AWS, GCP, Azure 등 클라우드 인프라에 대한 기본적인 이해가 있으신 분
•해야 할 일이나 문제를 스스로 찾고 빠르게 해결하는 것을 즐기는 분을 찾습니다.
•멀티 클라우드, 인프라 환경에 대한 서비스 및 구축 경험이 있으신 분
•Python 기반의 백엔드 개발 경험이 있으신 분
•pytorch 등 ML Framework 활용한 이미지 처리 경험
•5년 이상의 경력, 혹은 이에 준하는 경험이 있으신 분
•클라우드 서비스(AWS) 사용 경험이 있으신 분


# Chat GPT Open API 활용 방안


- 파이썬으로 ChatGPT API를 호출
- 프롬프팅을 통해 문장 요약문 생성기 만들기.
- ex) "채용 공고들의 자격 요건과 우대사항 리스트들을 모아서 보여줄게. 취업 준비자들을 위해 각 항목을 10개씩 요약해줘."
- (gpt-3.5 Turbo 기준) 토큰 당 입력: 0.001달러  출력ㅣ: 0.002달러
- 1,000토큰은 영단어 약 750단어 (1토큰: 약 0.75단어)